This notebook is designed to run in a IBM Watson Studio default runtime (NOT the Watson Studio Apache Spark Runtime as the default runtime with 1 vCPU is free of charge). Therefore, we install Apache Spark in local mode for test purposes only. Please don't use it in production.

In case you are facing issues, please read the following two documents first:

https://github.com/IBM/skillsnetwork/wiki/Environment-Setup

https://github.com/IBM/skillsnetwork/wiki/FAQ

Then, please feel free to ask:

https://coursera.org/learn/machine-learning-big-data-apache-spark/discussions/all

Please make sure to follow the guidelines before asking a question:

https://github.com/IBM/skillsnetwork/wiki/FAQ#im-feeling-lost-and-confused-please-help-me


If running outside Watson Studio, this should work as well. In case you are running in an Apache Spark context outside Watson Studio, please remove the Apache Spark setup in the first notebook cells.

In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')


In [2]:
#!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 10.0MB/s eta 0:00:01
     |████████████████████████████████| 204kB 39.4MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [4]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

Welcome to exercise two of “Apache Spark for Scalable Machine Learning on BigData”. In this exercise you’ll apply the basics of functional and parallel programming. 

Again, please use the following two links for your reference:
https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD
https://spark.apache.org/docs/latest/rdd-programming-guide.html

Let’s actually create a python function which decides whether a value is greater than 50 (True) or not (False).

In [5]:
def gt50(i):
    if i > 50:
        return True
    else:
        return False

In [6]:
print(gt50(4))
print(gt50(51))

False
True


Let’s simplify this function

In [7]:
def gt50(i):
    return i > 50

In [9]:
print(gt50(4))
print(gt50(51))

False
True


Now let’s use the lambda notation to define the function.

In [10]:
gt50 = lambda i: i > 50

In [11]:
print(gt50(4))
print(gt50(51))

False
True


In [15]:
#let's shuffle our list to make it a bit more interesting
from random import shuffle
l = list(range(100))
print(l)
shuffle(l)
print(l)
rdd = sc.parallelize(l)
print(rdd)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[15, 49, 91, 6, 14, 21, 44, 93, 20, 12, 48, 13, 33, 73, 83, 46, 62, 61, 66, 0, 57, 63, 26, 78, 76, 71, 56, 59, 60, 40, 3, 88, 7, 10, 38, 29, 28, 80, 50, 94, 53, 89, 9, 27, 77, 68, 85, 22, 11, 72, 79, 4, 36, 67, 35, 34, 52, 65, 98, 96, 87, 19, 47, 51, 90, 64, 86, 54, 1, 39, 16, 84, 92, 17, 42, 18, 69, 32, 75, 45, 74, 97, 95, 37, 99, 2, 41, 5, 70, 25, 30, 31, 82, 24, 55, 58, 8, 23, 43, 81]
ParallelCollectionRDD[3] at parallelize at PythonRDD.scala:195


Let’s filter values from our list which are equals or less than 50 by applying our “gt50” function to the list using the “filter” function. Note that by calling the “collect” function, all elements are returned to the Apache Spark Driver. This is not a good idea for BigData, please use “.sample(10,0.1).collect()” or “take(n)” instead.

In [16]:
rdd.filter(gt50).collect()

[91,
 93,
 73,
 83,
 62,
 61,
 66,
 57,
 63,
 78,
 76,
 71,
 56,
 59,
 60,
 88,
 80,
 94,
 53,
 89,
 77,
 68,
 85,
 72,
 79,
 67,
 52,
 65,
 98,
 96,
 87,
 51,
 90,
 64,
 86,
 54,
 84,
 92,
 69,
 75,
 74,
 97,
 95,
 99,
 70,
 82,
 55,
 58,
 81]

We can also use the lambda function directly.

In [17]:
rdd.filter(lambda i: i > 50).collect()

[91,
 93,
 73,
 83,
 62,
 61,
 66,
 57,
 63,
 78,
 76,
 71,
 56,
 59,
 60,
 88,
 80,
 94,
 53,
 89,
 77,
 68,
 85,
 72,
 79,
 67,
 52,
 65,
 98,
 96,
 87,
 51,
 90,
 64,
 86,
 54,
 84,
 92,
 69,
 75,
 74,
 97,
 95,
 99,
 70,
 82,
 55,
 58,
 81]

Let’s consider the same list of integers. Now we want to compute the sum for elements in that list which are greater than 50 but less than 75. Please implement the missing parts. 

In [21]:
# Method 1
rdd.filter(lambda x: x > 50).filter(lambda x: x<75 ).sum()

1500

In [22]:
# Method 2
rdd.filter(lambda x: x > 50 and x < 75).sum()

1500

You should see "1500" as answer. Now we want to know the sum of all elements. Please again, have a look at the API documentation and complete the code below in order to get the sum.